# Demo of best_autodiff

In [1]:
# import packages
import numpy as np
import sys 
from best_autodiff import Forward, Reverse
from best_autodiff import functions as ff
from best_autodiff import rfunctions as rf

## Forward Mode Demo

In [2]:
## R -> R: scalar case (1 function, 1 input)
def f(x):
    return ff.sin(x[0])

# instantiate forward mode object for function f that has 1 input and 1 output
forward = Forward(f,1,1)

# get value and jacobian of function at x = 0
value, jacobian = forward(0)
print(f'Value: {value}')
print(f'Jacobian: {jacobian}')

Value: 0.0
Jacobian: 1.0


In [3]:
## Rm -> R: vector case (1 function, muliple inputs)
def f(x):
    return x[0]*x[1]

# instantiate forward mode object for function f with multiple inputs
forward = Forward(f,2,1)

# get value and jacobian at x = [4,5]
value, jacobian = forward(np.array([4,5]))

# can also get partial derivatives with seed vectors
partial_x0 = forward.get_jacobian(np.array([4,5]), np.array([1,0]))
partial_x1 = forward.get_jacobian(np.array([4,5]), np.array([0,1]))
print(f'Value: {value}')
print(f'Jacobian: {jacobian}')
print(f'Partial derivative of x0: {partial_x0}')
print(f'Partial derivative of x1: {partial_x1}')

Value: [20]
Jacobian: [[5 4]]
Partial derivative of x0: [[5 0]]
Partial derivative of x1: [[0 4]]


In [4]:
## Rm -> Rn: multiple functions (multiple functions, 1 or multiple inputs)
def f1(x):
    return ff.sin(x[0])
def f2(x):
    return x[0]**2 + x[1]**2

# instantiate forward mode object for functions [f1,f2] that have multiple inputs x = [x0,x1]
forward = Forward([f1,f2],2,2)

# get function value and jacobian at x = [0,4]
value, jacobian = forward(np.array([0,4]))
print(f'Value:\n {value}')
print(f'Jacobian:\n {jacobian}')

# can also get weighted partial derivatives with weighted seed vector
# use same seed for each function
partials_weighted_same = forward.get_jacobian(np.array([0,4]), np.array([2,1/2]))
# use different seed for each function
partial_weighted_different = forward.get_jacobian(np.array([0,4]), np.array([[2,1/2],[1,0]]))
print(f'Weighted partial derivatives using the same seed:\n {partials_weighted_same}')
print(f'Weighted partial derivatives using different seeds:\n {partial_weighted_different}')

Value:
 [ 0. 16.]
Jacobian:
 [[1. 0.]
 [0. 8.]]
Weighted partial derivatives using the same seed:
 [[2. 0.]
 [0. 4.]]
Weighted partial derivatives using different seeds:
 [[2. 0.]
 [0. 0.]]


## Reverse Mode Demo

In [5]:
# R -> R
def f(x):
    return rf.sin(x[0])**3 + rf.sqrt(rf.cos(x[0]))
x = 1
reverse = Reverse(f, 1, 1)
value, jacobian = reverse(x)
print(f'Value: {value}')
print(f'Jacobian: {jacobian}')

Value: 1.3308758237356713
Jacobian: 0.5753328132280636


In [6]:
# R -> Rn
def f0(x):
    return x[0]**3
def f1(x):
    return x[0]
def f2(x):
    return rf.logistic(x[0])

f = [f0, f1, f2]
x = 1
reverse = Reverse(f, 1, 3) # 1 input, 3 outputs
value, jacobian = reverse(x)
print(f'Value: {value}')
print(f'Jacobian: \n {jacobian}')

Value: [1.         1.         0.73105858]
Jacobian: 
 [[3.        ]
 [1.        ]
 [0.19661193]]


In [7]:
# Rm -> R
def f(x):
    return (rf.sqrt(x[0])/rf.log(x[1]))*x[0]
x = [5, 6]
reverse = Reverse(f, 2, 1) # 2 inputs, 1 output
value, jacobian = reverse(x)
print(f'Value: {value}')
print(f'Jacobian: {jacobian}')

Value: [6.2398665]
Jacobian: [[ 1.87195995 -0.58042263]]


In [8]:
# Rm -> Rn
def f1(x):
    return rf.exp(-(rf.sin(x[0])-rf.cos(x[1]))**2)

def f2(x):
    return rf.sin(-rf.log(x[0])**2+rf.tan(x[2]))
f  = [f1, f2]
x = [1, 1, 1]
reverse = Reverse(f, 3, 2) # 3 inputs, 2 outputs
value, jacobian = reverse(x)
print(f'Value: {value}')
print(f'Jacobian: \n {jacobian}')

Value: [0.91328931 0.99991037]
Jacobian: 
 [[-0.29722477 -0.46290015  0.        ]
 [ 0.          0.          0.04586154]]
